# Transfer Learning

### Aufgabe
_Wählen Sie nun 5 andere Hunderassen aus. Nutzen Sie Transfer Learning zur Klassifikation dieser
Rassen, d.h. laden Sie die Gewichte aus Ihrem in Teilaufgabe B5 trainierten Modell als initiale
Gewichte für das neue Training. Beschreiben Sie ihr Vorgehen im Kurzreport und setzen Sie die
Evaluation dieses erneuten Trainings in Relation zu den Evaluationsergebnissen ihres ursprünglichen
Modells._

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

%matplotlib inline

from pathlib import Path
import sys
import os

notebook_dir = Path().resolve()
sys.path.append(str(notebook_dir.parent))

from setup import setup_dataset

# 5 andere Hunderassen
labels = [0, 2, 77, 84, 119]
train_dataset, test_dataset, metadata = setup_dataset('../dataset', labels=labels)

verification_image_dir = '../dataset/verification'
model_folder = 'models/'
model_filename = 'dog_classifier.h5'
weights_filename = 'model.weights.h5'

In [9]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(labels), activation='softmax')  # Number of classes
])

model.load_weights(os.path.join(model_folder, weights_filename))